### **REGLA COMPUESTA DE SIMPSON UTILIZANDO DASK**

In [1]:
import dask
import math
import multiprocessing
import time
from concurrent.futures import ProcessPoolExecutor
from dask.distributed import Client
from scipy.integrate import quad

In [2]:
# Definición de los parámetros a utilizar en ambas formas: secuencial y paralelo

f = lambda x: math.exp(-x**2) #función a integrar
a = 0 #extremo izquierdo del intervalo en donde se desea calcular la integral
b = 1 #extremo derecho del intervalo en donde se desea calcular la integral
n = 4*10**6 #número de subintervalos
h_hat = (b-a)/n #tamaño del subintervalo

In [3]:
# Valor real de la integral de la función

obj, err = quad(f,a,b)
obj

0.7468241328124271

In [4]:
# Creación del "distributed scheduler" a través de Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:33443 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.23 GB


**FORMA SECUENCIAL**

In [5]:
# Variables auxiliares para la creación de nodos
begin = 1
end = 2*n

In [6]:
'''
# Prueba para creación de nodos impares, es decir, los que se multiplican por 4
for i in range(begin,end,2):
    print(i)
'''

1
3
5
7
9
11
13
15


In [7]:
'''
# Prueba para creación de nodos pares, es decir, los que se multiplican por 2
for i in range(begin+1,end,2):
    print(i)
'''

2
4
6
8
10
12
14


In [6]:
def Simpson_compuesta(f,a,b,n):
    
    '''
Calcula la aproximación numérica utilizando al regla compuesta de Simpson.
Los nodos son generados a través de la fórmula: x_i = a + i/2 * h_hat, en donde:
i = 0,1,2,...,2n y h_hat = (b-a)/n


Args:
f (expresión lambda): función lambda a integrar
a(int): extremo izquierdo del intervalo en donde se desea evaluar la integral
b(int): extremo derecho del intervalo en donde se desea evaluar la integral
n(int): número de subintervalos, debe ser un número par

Return:
Simpson_compuesta(float)
''' 
    begin = 1
    end = 2*n
    
    h_hat = (b-a)/n
    
    sum2 = 0
    sum4 = 0
    
    # el siguiente loop considera las sumas de los términos que son multiplicados por 2
    for i in range(begin+1,end,2):
        x = a + i/2 * h_hat
        sum2 += f(x)
    
    sum2 = 2 *sum2
    
    # el siguiente loop considera las sumas de los términos que son multiplicados por 4
    for i in range(begin,end,2):
        x = a + i/2 * h_hat
        sum4 += f(x)
        
    sum4 = 4 * sum4
    
    return h_hat/6 * (f(a) + f(b) + sum2 + sum4)

In [7]:
'''
Calcula el tiempo de ejecución en segundos de una función utilizando el método time.time()

Arg:
funcion: función a la que se desea calcular tiempo de ejecución

Return:
tiempo_ejecucion(float)
'''

def calcular_tiempo(funcion):
    start_time = time.time()
    aprox = f
    end_time = time.time()
    tiempo = end_time - start_time
    print('Tiempo de ejecución de Simpson Compuesta: ',tiempo)

In [8]:
calcular_tiempo(Simpson_compuesta(f,a,b,n))

Tiempo de ejecución de Simpson Compuesta:  4.76837158203125e-07


In [9]:
%%time
aprox = Simpson_compuesta(f,a,b,n)

CPU times: user 3.04 s, sys: 18.9 ms, total: 3.05 s
Wall time: 3.03 s


In [10]:
'''
Calcula el error relativo entre el valor real y el valor aproximadao

Arg:S
aprox(float): valor obtenido en la aproximación numérica
obj (float): valor real 

Return:
error_relativo(float)
'''

def error_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj)

In [11]:
error_relativo(aprox,obj)

5.946369303542584e-16

**FORMA PARALELO**

In [12]:
# Definición de parámetros utilizados únicamente en la forma paralelo


p = multiprocessing.cpu_count() #número de CPUs en el sistema
ns_p = int(n/p) #número de subintervalos por proceso, si n (número de subintervalos) no es divisible por p, entonces se toma int(n/p) y se redefine n
n = p*ns_p # re-definición de n conforme al número de subintervalos por proceso
h_hat = (b-a)/n # re-definición del tamaño de los intervalos conforme al nuevo valor de n


In [13]:
print("número de CPU's (p):",p)
print("número de subintervalos (n):",n)
print("número de subintervalos por CPU (ns_p):",ns_p)

número de CPU's (p): 4
número de subintervalos (n): 4000000
número de subintervalos por CPU (ns_p): 1000000


In [16]:
'''
# Prueba de asignación de nodos pares(los que se multiplican por 4) a cada proceso 

for i in range(p):
    begin = i*ns_p + 1
    end = begin -1 + ns_p
    print('CPU:',i)
    print('  desde subintervalo:',begin)
    print('  hasta:',end)
    for k in range(2*begin, 2*end+1, 2):
        print('-----nodo:',k)
'''

CPU: 0
  desde subintervalo: 1
  hasta: 2
-----nodo: 2
-----nodo: 4
CPU: 1
  desde subintervalo: 3
  hasta: 4
-----nodo: 6
-----nodo: 8
CPU: 2
  desde subintervalo: 5
  hasta: 6
-----nodo: 10
-----nodo: 12
CPU: 3
  desde subintervalo: 7
  hasta: 8
-----nodo: 14
-----nodo: 16


In [17]:
'''
# Prueba de asignación de nodos impares (multiplicados por 4) a cada proceso

for i in range(p):
    begin = i*ns_p + 1
    end = begin -1 + ns_p
    print('CPU:',i)
    print('  desde subintervalo:',begin)
    print('  hasta:',end)
    for k in range(2*begin-1, 2*end, 2):
        print('-----nodo:',k)
'''

CPU: 0
  desde subintervalo: 1
  hasta: 2
-----nodo: 1
-----nodo: 3
CPU: 1
  desde subintervalo: 3
  hasta: 4
-----nodo: 5
-----nodo: 7
CPU: 2
  desde subintervalo: 5
  hasta: 6
-----nodo: 9
-----nodo: 11
CPU: 3
  desde subintervalo: 7
  hasta: 8
-----nodo: 13
-----nodo: 15


In [14]:
'''
Calcula la aproximación numérica de los últimos tres términos de la regla compuesta de Simpson
Los nodos son generados a través de la fórmula: x_i = a + i/2 * h_hat, en donde:
i = 1,2,...,2n y h_hat = (b-a)/n


Args:
mi_id(int): id de cada CPU en el sistema, se obtiene a través de range(multiprocessing.count())
f (expresión lambda): función lambda a integrar
a(int): extremo izquierdo del intervalo en donde se desea evaluar la integral
b(int): extremo derecho del intervalo en donde se desea evaluar la integral
n(int): número de subintervalos, debe ser un número par
ns_p(int): número de subintervalos por proceso


Return:
Simpson_compuesta(float)
''' 
def Simpson_truncada(mi_id,f,a,b,n,ns_p):
    
    #ns_p = int(n/p) #número de subintervalos por proceso, si n (número de subintervalos) no es divisible por p, entonces se toma int(n/p) y se redefine n
    #n = p*ns_p
    
    h_hat = (b-a)/n
    
    begin = mi_id*ns_p + 1
    end = begin -1 + ns_p
    
    sum2 = 0
    sum4 = 0
    
    # el siguiente loop considera las sumas de los términos que son multiplicados por 2
    for i in range(2*begin, 2*end+1, 2):
        x = a + i/2 * h_hat
        sum2 += f(x)
    
    sum2 = 2 *sum2
    
    # el siguiente loop considera las sumas de los términos que son multiplicados por 4
    for i in range(2*begin-1, 2*end, 2):
        x = a + i/2 * h_hat
        sum4 += f(x)
        
    sum4 = 4 * sum4
    
    return sum2 + sum4

In [15]:
'''
Calcula la aproximación numérica de la regla compuesta de Simpson

Args:
cl: Client obtenido a través de Client()
'''

def Simpson_compuesta_paralelo(cl):
    futures_evalua = client.map(Simpson_truncada,range(p),
                           **{'f':f,
                              'a':a,
                              'b':b,
                              'n':n,
                              'ns_p':ns_p}
                           )

    results = client.gather(futures_evalua)
    return h_hat/6 * (f(a) - f(b) + sum(results))

In [16]:
calcular_tiempo(Simpson_compuesta_paralelo(client))

Tiempo de ejecución de Simpson Compuesta:  2.384185791015625e-07


In [17]:
%%time
aprox = Simpson_compuesta_paralelo(client)

CPU times: user 124 ms, sys: 16.4 ms, total: 140 ms
Wall time: 2.15 s


In [18]:
error_relativo(aprox,obj)

1.516324172403359e-14

In [19]:
client.close()

Se realizaron pruebas utilizando tanto la forma secuencial como la forma en paralelo de la regla compuesta de Simpson; y se varió el número de nodos obteniéndose los siguientes resultados:


| Tipo       | n     | time.time() | user    | sys     | total   | wall   |
|------------|-------|-------------|---------|---------|---------|--------|
| Secuencial | 8     | 0.23 $\mu s$     | 78 $\mu s$     | 0       | 78 $\mu s$      | 91.6 $\mu s$   |
| Paralelo   | 8     | 0.23 $\mu s$     | 40.1 $ms$ | 2.12 $ms$ | 42.2 $ms$ | 55 $ms$  |
| Secuencial | 4x10⁶ | 0.47 $\mu s$     | 3.12 $s$  | 0     | 3.12 $s$  | 31 $s$   |
| Paralelo   | 4x10⁶ | 0.23 $\mu s$     | 148 $ms$  | 12.6 $ms$ | 160 $ms$  | 2.19 $s$ |


Se puede concluir que al utilizar un número pequeño de subintervalos, la forma Secuencial es más rápida que la forma en Paralelo; sin embargo a aumentar el número de subintervalos a un orden de millones, el comportamiento se invierte es decir, la forma en Paralelo es más rápida que la forma Secuencial.

Adicional, al considerar 8 subintervalos, se obtiene un error relativo del orden de $10^{-7}$; mientras que al aumentar el número de subintervalos a $4x10^{6}$, el error relativo disminuye a un orden de $10^{-14}$.